In [7]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [2]:
w, h = 20, 20
base_path = os.path.join('dataset', '*/*.npz')

file_list = glob(base_path)

x_data, y_data = [], []
for file_path in tqdm(file_list):
    data = np.load(file_path)
    x_data.extend(data['inputs'])
    y_data.extend(data['outputs'])

x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
y_data = np.array(y_data, np.float32).reshape((-1, h * w))

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

100%|██████████| 10224/10224 [00:11<00:00, 884.09it/s]


(4515129, 20, 20, 1) (4515129, 400)
(1128783, 20, 20, 1) (1128783, 400)


In [3]:
model = models.Sequential([
    layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=(h, w, 1)),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(256, 7, activation='relu', padding='same'),
    layers.Conv2D(128, 7, activation='relu', padding='same'),
    layers.Conv2D(64, 7, activation='relu', padding='same'),
    layers.Conv2D(1, 1, activation=None, padding='same'),
    layers.Reshape((h * w,)),
    layers.Activation('sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 20, 64)        3200      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 128)       401536    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 256)       1605888   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       1605760   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 64)        401472    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 1)         65        
_________________________________________________________________
reshape (Reshape)            (None, 400)               0

- kernel 7,5,3,3,3: 53.3
- kernel all 7: 56.12
- kernel all 7, double kernel:

In [ ]:
start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('models', exist_ok=True)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[
        ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

Epoch 1/10
17637/17638 [============================>.] - ETA: 0s - loss: 0.0049 - acc: 0.6081
Epoch 00001: val_acc improved from -inf to 0.58375, saving model to ./models/20201213_202430.h5
17638/17638 [==============================] - 1091s 62ms/step - loss: 0.0049 - acc: 0.6081 - val_loss: 0.0052 - val_acc: 0.5837
Epoch 2/10
17637/17638 [============================>.] - ETA: 0s - loss: 0.0048 - acc: 0.6206
Epoch 00002: val_acc improved from 0.58375 to 0.58817, saving model to ./models/20201213_202430.h5
17638/17638 [==============================] - 1095s 62ms/step - loss: 0.0048 - acc: 0.6206 - val_loss: 0.0052 - val_acc: 0.5882
Epoch 3/10
17637/17638 [============================>.] - ETA: 0s - loss: 0.0046 - acc: 0.6330
Epoch 00003: val_acc improved from 0.58817 to 0.59378, saving model to ./models/20201213_202430.h5
17638/17638 [==============================] - 1096s 62ms/step - loss: 0.0046 - acc: 0.6330 - val_loss: 0.0052 - val_acc: 0.5938
Epoch 4/10
17637/17638 [==========

In [38]:
i = 0

for y in range(h):
    for x in range(w):
        print('%2d' % x_val[i][y, x], end='')
    print()

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0-1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0-1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [39]:
y_pred = model.predict(np.expand_dims(x_val[i], axis=0)).squeeze()
y_pred = y_pred.reshape((h, w))

y, x = np.unravel_index(np.argmax(y_pred), y_pred.shape)

print(x, y, y_pred[y, x])

6 9 0.99919564
